In [1]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: data.table

Loading required package: parallel

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings


Loading required package: primes

Loading required 

In [2]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

In [3]:
# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle
  )

  return( drealidad )
}

In [4]:
# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}

In [5]:
# carpeta de trabajo
PARAM <- list()
PARAM$experimento <- 7001

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [6]:
library(yaml)

PARAM <- yaml::read_yaml("PARAM.yml")

# eliminar solo la parte que no querés
PARAM$hyperparametertuning <- NULL


#Training

In [7]:
dataset <-fread("/content/dataset_1110_lag1y2_maxminmean_aguinaldo.csv.gz",stringsAsFactors= TRUE)

In [8]:
ncol(dataset)

[1] 2122

In [9]:
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

In [10]:
PARAM$train_final <-c(202101,202102,202103,202104)
PARAM$future <-202106

dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

clase_ternaria,N
<fct>,<int>
CONTINUA,642824
BAJA+2,3938
BAJA+1,3447


In [11]:
campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

In [12]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

In [13]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$num_threads
[1] 4

$seed
[1] 505051

$max_depth
[1] -1

$min_gain_to_split
[1] 0.2165657

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$max_bin
[1] 31

$bagging_fraction
[1] 0.8034078

$bagging_freq
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] TRUE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 4980

$learning_rate
[1] 0.0101627

$feature_fraction
[1] 0.5204208

$num_leaves
[1] 9998

$min_data_in_leaf
[1] 51

In [14]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

In [15]:
param_normalizado$num_threads <-1
param_normalizado

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$num_threads
[1] 1

$seed
[1] 505051

$max_depth
[1] -1

$min_gain_to_split
[1] 0.2165657

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$max_bin
[1] 31

$bagging_fraction
[1] 0.8034078

$bagging_freq
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] TRUE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 4980

$learning_rate
[1] 0.0101627

$feature_fraction
[1] 0.5204208

$num_leaves
[1] 9998

$min_data_in_leaf
[1] 255

In [16]:
Sys.time()

[1] "2025-10-12 05:14:50 UTC"

In [17]:
# #-------------------------------------------------------------
# # CONFIGURACIÓN
# #-------------------------------------------------------------
# semillas <- c(202021, 505051, 707071, 808081, 909091, 101267, 106121, 106123, 106129, 111263)

# # tabla para guardar los resultados
# ganancias <- data.table(
#   semilla = semillas,
#   mejor_total = NA_real_
# )

# #-------------------------------------------------------------
# # LOOP DE ENTRENAMIENTO CON DISTINTAS SEMILLAS
# #-------------------------------------------------------------
# for (i in seq_along(semillas)) {

#   cat("\nEntrenando modelo con semilla:", semillas[i], "...\n")

#   # parámetros específicos de la semilla
#   param_i <- param_normalizado
#   param_i$seed <- semillas[i]
#   #param_i$num_threads <- 8

#   modelo_i <- lgb.train(
#     data = dtrain_final,
#     param = param_i
#   )

#   #-------------------------------------------------------------
#   # PREDICCIÓN
#   #-------------------------------------------------------------
#   dfuture <- dataset[foto_mes %in% PARAM$future]
#   prediccion_i <- predict(
#     modelo_i,
#     data.matrix(dfuture[, campos_buenos, with = FALSE])
#   )

#   drealidad <- realidad_inicializar(dfuture, PARAM)

#   tb_prediccion <- copy(dfuture[, list(numero_de_cliente, foto_mes)])
#   tb_prediccion[, prob := prediccion_i]
#   setorder(tb_prediccion, -prob)

#   #-------------------------------------------------------------
#   # BUSCAR LA MÁXIMA GANANCIA ENTRE CORTES
#   #-------------------------------------------------------------
#   mejor_total <- -Inf
#   mejor_envios <- NA_integer_

#   for (envios in PARAM$cortes) {
#     tb_prediccion[, Predicted := 0L]
#     tb_prediccion[1:envios, Predicted := 1L]

#     res <- realidad_evaluar(drealidad, tb_prediccion)

#     if (res$total > mejor_total) {
#       mejor_total <- res$total
#       mejor_envios <- envios
#     }
#   }

#   set(ganancias, i = i, j = "mejor_total", value = mejor_total)

#   cat(
#     "Semilla=", semillas[i],
#     " | Mejor corte=", mejor_envios,
#     " | Ganancia total=", mejor_total, "\n"
#   )
# }

# #-------------------------------------------------------------
# # RESULTADOS FINALES
# #-------------------------------------------------------------
# cat("\nResumen de ganancias máximas por semilla:\n")
# print(ganancias)

# cat("\nGanancia promedio total entre semillas:\n")
# print(mean(ganancias$mejor_total, na.rm = TRUE))

In [18]:
Sys.time()

[1] "2025-10-12 05:14:50 UTC"

In [ ]:
# --- 4. GENERAR PREDICCIONES FINALES (ENSAMBLE DE PRODUCCIÓN) ---
semillas_ensemble <- c(202021, 505051, 707071, 808081, 909091, 101267, 106121, 106123, 106129, 111263)


dfuture <- dataset[foto_mes %in% PARAM$future]

archivo_progreso_prod <- "ensemble_predicciones_final_temp.csv" # Nuevo nombre de archivo de progreso

# Cargamos el progreso si ya existe
if (file.exists(archivo_progreso_prod)) {
  predicciones_ensemble_final <- fread(archivo_progreso_prod)
} else {
  predicciones_ensemble_final <- dfuture[, .(numero_de_cliente, foto_mes)]
}

dtrain <- lgb.Dataset(data = data.matrix(dataset_train[, campos_buenos, with = FALSE]),
                      label = dataset_train[, clase01])

for (semilla_actual in semillas_ensemble) {
  nombre_col_pred <- paste0("pred_", semilla_actual)

  if (nombre_col_pred %in% colnames(predicciones_ensemble_final)) {
    cat("Saltando semilla (ya existe en producción):", semilla_actual, "\n")
    next
  }

  cat("\nCorriendo ensamble de PRODUCCIÓN para semilla:", semilla_actual, "\n")
  param_final <- modifyList(PARAM$lgbm$param_fijos, PARAM$out$lgbm$mejores_hiperparametros)
  param_final$seed <- semilla_actual
  modelo <- lgb.train(data = dtrain, param = param_final)
  prediccion_actual <- predict(modelo, data.matrix(dfuture[, campos_buenos, with = FALSE]))
  predicciones_ensemble_final[, (nombre_col_pred) := prediccion_actual]

  fwrite(predicciones_ensemble_final, archivo_progreso_prod)
  cat("Progreso de producción guardado para semilla:", semilla_actual, "\n")
}

columnas_predicciones <- grep("pred_", colnames(predicciones_ensemble_final), value = TRUE)
predicciones_ensemble_final[, prob_promedio := rowMeans(.SD), .SDcols = columnas_predicciones]
cat("\nPredicción final de ensamble para Junio generada con éxito.\n")

In [ ]:
Sys.time()

In [ ]:
range(predicciones_finales[[9]]$prob)
range(pred_final_202106$prob_promedio)


In [ ]:
# PARAM$experimento
# PARAM$semilla_primigenia
# PARAM$train
# PARAM$train_final
# PARAM$future
# PARAM$semilla_kaggle
# PARAM$cortes
# PARAM$trainingstrategy$undersampling
# PARAM$lgbm$param_fijos